In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn
from functions import *

In [2]:
url1 = 'data/df_final_demo.txt'
url2 = 'data/df_final_experiment_clients.txt'
url3 = 'data/df_final_web_data_pt_1.txt'
url4 = 'data/df_final_web_data_pt_2.txt'

df_final_demo = pd.read_csv(url1)
df_final_experiment_clients = pd.read_csv(url2)
df_web_data_1 = pd.read_csv(url3)
df_web_data_2 = pd.read_csv(url4)


In [3]:
df_final_demo.head(5)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [4]:
df_final_experiment_clients.tail(5)

,client_id,Variation
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN
70608,9895983,NaN


In [5]:
# Create list with client_id for compare each df

list_id_final_demo = df_final_demo['client_id']

list_id_final_exp = df_final_experiment_clients['client_id']

common_clients_id = set(list_id_final_demo).intersection(set(list_id_final_exp))
demo_clients_id = set(list_id_final_demo).difference(set(list_id_final_exp))
exp_clients_id = set(list_id_final_exp).difference(set(list_id_final_demo))

{
    "Commons ID": len(common_clients_id),
    "Demo ID": len(demo_clients_id),
    "Exp ID": len(exp_clients_id)
}


{'Commons ID': 70609, 'Demo ID': 0, 'Exp ID': 0}

In [6]:
## Searching Nan Values in Variaton

nan_var = df_final_experiment_clients[df_final_experiment_clients['Variation'].isna()]

nan_var_count = nan_var.shape[0]
nan_var.tail(10), nan_var_count # 20109 NaN


(       client_id Variation
 70599    5217934       NaN
 70600    1020453       NaN
 70601    9722009       NaN
 70602    3897324       NaN
 70603     687702       NaN
 70604    2443347       NaN
 70605    8788427       NaN
 70606     266828       NaN
 70607    1266421       NaN
 70608    9895983       NaN,
 20109)

In [7]:
df_final_experiment_clients['Variation'].value_counts()

df_final_experiment_clients.shape

(70609, 2)

In [8]:
## Check the clients_id on web_data

temp_web_data = pd.concat([df_web_data_1, df_web_data_2], ignore_index=True)

nan_var_id = nan_var['client_id']
clients_id_web = temp_web_data[temp_web_data['client_id'].isin(nan_var_id)]

clients_id_web['client_id'].nunique() ## The NaN ID exists on Web Data



20109

In [9]:
total_rows_web = len(df_web_data_1['client_id']) + len(df_web_data_2['client_id'])
temp_web_data.shape, total_rows_web

((755405, 5), 755405)

In [10]:
merged_web_data = pd.merge(
    temp_web_data,
    df_final_experiment_clients[['client_id', 'Variation']],
    on = 'client_id',
    how = 'left'
)

merged_web_data['Variation'].value_counts()

Variation
Test       177847
Control    143462
Name: count, dtype: int64

## Try to substitute NaN for real Value

In [11]:
merged_web_data['date_time'] = pd.to_datetime(merged_web_data['date_time'])

nan_clients = merged_web_data[merged_web_data['Variation'].isna()]['client_id'].unique()

nan_clients_web = merged_web_data[merged_web_data['client_id'].isin(nan_clients)]
test_clients_web = merged_web_data[merged_web_data['Variation'] == 'Test']
control_clients_web = merged_web_data[merged_web_data['Variation'] == 'Control']

nan_steps = nan_clients_web['process_step'].value_counts()
test_steps = test_clients_web['process_step'].value_counts()
control_steps = control_clients_web['process_step'].value_counts()

{
   "NaN Variation by steps:": nan_steps,
   "Control Variation by steps:": control_steps,
   "Test Variation by steps:": test_steps,
}

{'NaN Variation by steps:': process_step
 start      139863
 step_1      94757
 step_2      76187
 step_3      63553
 confirm     59736
 Name: count, dtype: int64,
 'Control Variation by steps:': process_step
 start      47608
 step_1     29668
 step_2     25886
 step_3     22795
 confirm    17505
 Name: count, dtype: int64,
 'Test Variation by steps:': process_step
 start      56474
 step_1     38768
 step_2     30989
 step_3     25894
 confirm    25722
 Name: count, dtype: int64}

In [12]:
nan_client_steps_mean = nan_clients_web.groupby('client_id')['process_step'].count().mean()
control_client_steps_mean = control_clients_web.groupby('client_id')['process_step'].count().mean()
test_client_steps_mean = test_clients_web.groupby('client_id')['process_step'].count().mean()

nan_client_steps_mean, control_client_steps_mean, test_client_steps_mean

(np.float64(6.231907776677146),
 np.float64(6.096464388917219),
 np.float64(6.594741916345298))

No trend can be determined from the number of steps or the average. Let's look at the dates.

In [13]:


test_client_datetime = merged_web_data[merged_web_data['Variation'] == 'Test']['date_time'].describe()
control_client_datetime = merged_web_data[merged_web_data['Variation'] == 'Control']['date_time'].describe()
nan_client_datetime = merged_web_data[merged_web_data['client_id'].isin(nan_clients)]['date_time'].describe()

nan_client_datetime, 'Test-------', test_client_datetime, 'Control-------', control_client_datetime

(count                           434096
 mean     2017-05-21 03:57:17.316364800
 min                2017-03-15 00:03:03
 25%      2017-05-07 13:22:22.750000128
 50%                2017-06-01 11:05:00
 75%                2017-06-07 17:39:11
 max                2017-06-20 23:59:57
 Name: date_time, dtype: object,
 'Test-------',
 count                           177847
 mean     2017-04-18 07:31:33.564833536
 min                2017-03-15 00:43:23
 25%         2017-04-03 17:29:01.500000
 50%                2017-04-12 11:40:13
 75%         2017-04-24 13:09:25.500000
 max                2017-06-20 23:21:23
 Name: date_time, dtype: object,
 'Control-------',
 count                           143462
 mean     2017-04-22 06:20:59.694309376
 min                2017-03-15 00:19:28
 25%      2017-04-05 11:23:32.249999872
 50%         2017-04-12 21:17:38.500000
 75%                2017-05-07 20:48:26
 max                2017-06-20 23:57:06
 Name: date_time, dtype: object)

In [14]:
nan_clients_web.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
Variation               object
dtype: object

In [15]:
# Calculate session time per client
nan_client_connection_time = nan_clients_web.groupby('client_id')['date_time'].apply(calc_connection_time)
test_client_connection_time = test_clients_web.groupby('client_id')['date_time'].apply(calc_connection_time)
control_client_connection_time = control_clients_web.groupby('client_id')['date_time'].apply(calc_connection_time)

# Add to dataframe
nan_clients_web = nan_clients_web.merge(nan_client_connection_time.rename('connection_time'), on='client_id')
test_clients_web = test_clients_web.merge(test_client_connection_time.rename('connection_time'), on='client_id')
control_clients_web = control_clients_web.merge(control_client_connection_time.rename('connection_time'), on='client_id')

# Calculate mean time
mean_test_connection = test_clients_web['connection_time'].mean()
mean_control_connection = control_clients_web['connection_time'].mean()


In [17]:
print("Mean conection time (Test):", mean_test_connection)
print("Mean conection time (Control):", mean_control_connection)

Mean conection time (Test): 674133.1151158018
Mean conection time (Control): 717933.5775466674


In [ ]:
steps_per_client = nan_clients_web.groupby('client_id')['process_step'].count().reset_index()
steps_per_client.columns = ['client_id', 'step_count']

nan_clients_web = nan_clients_web.merge(steps_per_client, on='client_id')

In [48]:
nan_clients_web['step_count'] = nan_clients_web.groupby('client_id')['process_step'].transform('count')

# Calculate similarities based on completed steps

nan_clients_web['similarity_test_steps'] = np.maximum(0, 1 - abs(nan_clients_web['step_count'] - test_client_steps_mean) / test_client_steps_mean)
nan_clients_web['similarity_control_steps'] = np.maximum(0, 1 - abs(nan_clients_web['step_count'] - control_client_steps_mean) / control_client_steps_mean)

# Calculate similarities based on completed time

nan_clients_web['similarity_test_time'] = np.maximum(0, 1 - abs(nan_clients_web['connection_time'] - mean_test_connection) / mean_test_connection)
nan_clients_web['similarity_control_time'] = np.maximum(0, 1 - abs(nan_clients_web['connection_time'] - mean_control_connection) / mean_control_connection)

# Calculate prob based on steps and time
nan_clients_web['test_probability'] = (nan_clients_web['similarity_test_steps'] + nan_clients_web['similarity_test_time']) / 2
nan_clients_web['control_probability'] = 1 - nan_clients_web['test_probability']


       similarity_test_steps  similarity_test_time
count          434096.000000         434096.000000
mean                0.597056              0.089405
std                 0.319849              0.211994
min                 0.000000              0.000000
25%                 0.332004              0.000224
50%                 0.758180              0.000533
75%                 0.786912              0.002712
max                 0.938548              0.999737


In [49]:
# Asign type of client by proximity
nan_clients_web['assigned_group'] = np.where(nan_clients_web['test_probability'] > 0.5, 'Test', 'Control')

# Results
nan_clients_web[['client_id', 'test_probability', 'control_probability', 'assigned_group']]

print(nan_clients_web['assigned_group'].value_counts())


assigned_group
Control    398388
Test        35708
Name: count, dtype: int64


In [50]:
merged_web_data.shape

(755405, 6)

## Cositas de los testeos

In [16]:
confirmation_complete = merged_web_data[merged_web_data['process_step'] == 'confirm']

conversion_counts = confirmation_complete.groupby('Variation')['client_id'].nunique()

total_clients = merged_web_data.groupby('Variation')['client_id'].nunique()

total_clients.sum()


np.int64(50500)